In [440]:
import random
import numpy as np
import PDSVRP_instance
import copy

metodini "utilities"

In [441]:
def get_drone_customers(drone_tours):
    drone_customers = []
    for tour in drone_tours:
        for customer in tour:
            drone_customers.append(customer)

    return drone_customers

In [442]:
def get_truck_customers(truck_tours):
    truck_customers = []
    for tour in truck_tours:
        for customer in tour:
            truck_customers.append(customer)

    return truck_customers

In [443]:
def sort_by_angular(coordinates, c_drones, grad_seed):

    rad_seed = np.deg2rad(grad_seed)
    ref_point = coordinates[0]

    def polar_angle(coord):
        # Compute the angle in radians with respect to the reference point
        dx = coord[0] - ref_point[0]
        dy = coord[1] - ref_point[1]
        angle = np.arctan2(dy, dx)
        return angle

    def angular_distance(angle1, angle2):
        # Compute the absolute angular distance between two angles
        return min(abs(angle1 - angle2), 2 * np.pi - abs(angle1 - angle2))

    # Calculate the polar angles for all nodes in c_drones
    angles = [polar_angle(coordinates[i]) for i in c_drones]

    # Calculate the angular distance from grad_seed
    distances = [angular_distance(angle, rad_seed) for angle in angles]

    # Pair the indices with their angular distances
    indexed_distances = list(zip(c_drones, distances))

    # Sort based on angular distance
    indexed_distances.sort(key=lambda x: x[1])

    # Extract the sorted indices
    sorted_c_drones = [index for index, _ in indexed_distances]

    return sorted_c_drones

#test ok

In [444]:
def sort_by_euclidean_distance(distance_matrix, c_seed):
    distances = [distance_matrix[c_seed][i] for i in range(1, len(distance_matrix[c_seed]))]

    sorted_c = np.argsort(distances)
    sorted_c = [x + 1 for x in sorted_c]

    return sorted_c

# test ok

In [445]:
def remove_drone_customer(drone_tour, absent_customers, customer_to_remove):
    new_tour = drone_tour
    new_tour.remove(customer_to_remove)
    new_absent_vector = absent_customers + [customer_to_remove]

    return new_tour, new_absent_vector

#test ok

In [446]:
def max_string_length(truck_tours, L_max):
    average_tour_cardinality = len(get_truck_customers(truck_tours)) / (len(truck_tours))
    return min(average_tour_cardinality, L_max)

In [447]:
def n_strings_to_remove(c_average_removed, l_s_max):
    k_s_max = (4 * c_average_removed) / (1 + l_s_max) - 1
    k_s = int(random.uniform(1, k_s_max + 1))  
    
    return k_s

# test ok

In [448]:
def string_to_remove_length(l_s_max, truck_tour):
    l_t_max = min(len(truck_tour), l_s_max)
    l_t = int(random.uniform(1, l_t_max + 1))
    return l_t

# test ok

In [449]:
def sort_absent_customers(instance, solution, w1, w2, w3, w4, w5):
    
    absent_customers = solution[1]

    def random_sort(instance, solution, absent_customers):
        random.shuffle(absent_customers)
        return absent_customers
        
    def near_dp(instance, solution, absent_customers):
        absent_customers = sorted(absent_customers, key = lambda customer : instance.distances[0][customer])
        return absent_customers
        
    def far_dp(instance, solution, absent_customers):
        absent_customers = sorted(absent_customers, key = lambda customer : (1/instance.distances[0][customer]))
        return absent_customers
        
    def near_tr(instance, solution, absent_customers):
        truck_customers = get_truck_customers(solution[0][0])
        if (len(truck_customers) == 0):
            return random_sort(instance, solution, absent_customers)
        absent_customers = sorted(absent_customers, key = lambda customer : min(instance.distances[t_customer][customer] for t_customer in truck_customers))
        return absent_customers
        
        
    def far_tr(instance, solution, absent_customers):
        truck_customers = get_truck_customers(solution[0][0])
        if (len(truck_customers) == 0):
            return random_sort(instance, solution, absent_customers)
        absent_customers = sorted(absent_customers, key = lambda customer : 1/(min(instance.distances[t_customer][customer] for t_customer in truck_customers)))
        return absent_customers

    # Lista dei blocchi e dei relativi pesi
    sorting_methods = [random_sort, near_dp, far_dp, near_tr, far_tr]
    weights = [w1, w2, w3, w4, w5]

    selected_method = random.choices(sorting_methods, weights, k=1)[0]

    return selected_method(instance, solution, absent_customers)

In [450]:
def truck_tour_time(truck_travel_times, tour):
    print("truc travel times", len(truck_travel_times), len(truck_travel_times[0]))
    if (len(tour) == 0):
        return 0
    total_time = truck_travel_times[0][tour[0]]
    
    for i in range(len(tour) - 1):
        print("tour[i]", tour[i])

        print("distanze da i", truck_travel_times[tour[i]])

        total_time += truck_travel_times[tour[i]][tour[i + 1]]

           
    total_time += truck_travel_times[tour[-1]][0]
    
    return total_time


In [451]:
def drone_tour_time(drone_travel_times, tour):
   
    if (len(tour) == 0):
        return 0
    
    total_time = 0
    
    for i in range(len(tour)):
        total_time += (drone_travel_times[tour[i]]) * 2

   
    
    return total_time

In [452]:
def remove_string(truck_tour, string_length, customer, absent_customers):
    customer_index = next((index for index, c in enumerate(truck_tour) if c == customer), -1)
    if (len(truck_tour) < customer_index + string_length):
        truck_tour = truck_tour[:len(truck_tour)-string_length]
        absent_customers += [truck_tour[len(truck_tour)-string_length:]]

    else:
        truck_tour = truck_tour[:customer_index] + truck_tour[customer_index + string_length:]
        absent_customers += truck_tour[customer_index: customer_index + string_length]

    return truck_tour, absent_customers

In [453]:
def is_truck_tour_feasible(instance, tour):
    return (truck_tour_time(instance.t_t, tour) <= instance.T_t) and sum(instance.w[customer] for customer in tour) <= instance.Q_t

In [454]:
def feasible_truck_tour_positions_calculation(instance, truck_tours, customer): #in non-empty truck tours
    feasible_positions = []
    for i in range(len(truck_tours)):
        if len(truck_tours[i]) > 0:
            for j in range (len(truck_tours[i]) + 1): # lo posso aggiungere anche come ultimo
                new_tour = copy.deepcopy(truck_tours[i])
                new_tour.insert(j, customer)
                if (is_truck_tour_feasible(instance, new_tour)):
                     feasible_positions.append([0, i, j])

    return feasible_positions


In [455]:
def is_drone_tour_feasible(instance, tour):
    for customer in tour:
       if instance.w[customer] > instance.Q_d:
        return False 
    return (drone_tour_time(instance.t_d, tour) <= instance.T_d)

In [456]:
def is_drone_eligible(instance, drone_tours, customer):
    if instance.w[customer] > instance.Q_d:
        return False
    for tour in drone_tours:
        new_tour = copy.deepcopy(tour)
        new_tour.append(customer)
        if (drone_tour_time(instance.t_d, new_tour) <= instance.T_d):
            return True
    
    return False

In [457]:
def insert_customer(customer, pos, solution_):
    
    solution_[0][pos[0]][pos[1]].insert(pos[2], customer)
    
    return solution_

In [458]:
array = [0,1,2]
array.insert(1, 10)
array[3]

2

In [459]:
def largest_spatial_slack_drone(instance, drone_tours):
    chosen_drone = min(range(len(drone_tours)), key=lambda i: drone_tour_time(instance.t_d, drone_tours[i]))
    return chosen_drone

In [460]:
def cost(instance, solution):
    total_cost = 0
    for truck_tour in solution[0][0]:
        total_cost += truck_tour_time(instance.t_t, truck_tour) * instance.t_speed * instance.C_T

    for drone_tour in solution[0][1]:
        total_cost += drone_tour_time(instance.t_d, drone_tour) * instance.d_speed * instance.C_D


    return total_cost

In [461]:
def select_nearest_neighbors(n_nearest, distances_from_customer):
    indexed_array = [(value, idx) for idx, value in enumerate(distances_from_customer) if idx != 0]
    
    indexed_array.sort(key=lambda x: x[0])
    
    nearest_neighbors = [idx for value, idx in indexed_array[1:n_nearest + 1]] #the depot and the customer itself are escluded
    
    return nearest_neighbors

#test ok

In [462]:
def find_customer_in_vehicle_tours(tours, customer):
    for i, row in enumerate(tours):
        for j, element in enumerate(row):
            if element == customer:
                return [i, j]
    return None

In [463]:
def _2_opt_x(instance, solution, customer, neighbor, customer_tour_index):
    neighbor_index = find_customer_in_vehicle_tours(solution[0][0], neighbor)
    customer_index = [customer_tour_index, next((index for index, c in enumerate(solution[0][0][customer_tour_index]) if c == customer), -1)]
    new_tour_1 = [solution[0][0][neighbor_index[0]][:neighbor_index[1]]] + [solution[0][0][customer_index[0]][customer_index[1]:]]
    new_tour_2 = [solution[0][0][customer_index[0]][:customer_index[1]]] + [solution[0][0][neighbor_index[0]][neighbor_index[1]:]]
    if (is_truck_tour_feasible(instance, new_tour_1) and is_truck_tour_feasible(instance,new_tour_2)):
        solution[0][0][neighbor_index[0]] = new_tour_1
        solution[0][0][customer_index[0]] = new_tour_2

    return solution

In [464]:
def relocate(instance, solution, customer, neighbor, tour_index):
    #customer_index = next((index for index, c in enumerate(solution[0][0][tour_index]) if c == customer), -1)
    neighbor_index = next((index for index, c in enumerate(solution[0][0][tour_index]) if c == neighbor), -1)

    proposed_tour = copy.deepcopy(solution[0][0][tour_index])
    print("customer", customer)
    print("proposed tour", proposed_tour)
    proposed_tour.remove(customer)
    proposed_tour.insert(neighbor_index, customer)

    other_tour = copy.deepcopy(solution[0][0][tour_index])
    other_tour.remove(customer)
    other_tour.insert(neighbor_index + 1, customer)

    if (truck_tour_time(instance.t_t, proposed_tour) > truck_tour_time(instance.t_t, other_tour)):
        proposed_tour = other_tour

    if (is_truck_tour_feasible(instance, proposed_tour)):
        solution[0][0][tour_index] = proposed_tour
        
    return solution




In [465]:
def swap(instance, solution, customer, neighbor, tour_index):
    customer_index = next((index for index, c in enumerate(solution[0][0][tour_index]) if c == customer), -1)
    neighbor_index = next((index for index, c in enumerate(solution[0][0][tour_index]) if c == neighbor), -1)

    proposed_tour = copy.deepcopy(solution[0][0][tour_index])
    proposed_tour[customer_index] = neighbor
    proposed_tour[neighbor_index] = customer

    if (is_truck_tour_feasible(instance, proposed_tour)):
        solution[0][0][tour_index] = proposed_tour

    return solution

In [466]:
def _2_opt(instance, solution, customer, neighbor, tour_index):
    customer_index = next((index for index, c in enumerate(solution[0][0][tour_index]) if c == customer), -1)
    neighbor_index = next((index for index, c in enumerate(solution[0][0][tour_index]) if c == neighbor), -1)

    i = min(customer_index, neighbor_index)
    j = max(customer_index, neighbor_index)

    proposed_tour = copy.deepcopy(solution[0][0][tour_index])

    if (i == 0):
        proposed_tour[:j+1] = proposed_tour[j::-1]
        return solution

    

    proposed_tour[i:j+1] = proposed_tour[j:i-1:-1]

    if (is_truck_tour_feasible(instance, proposed_tour)):
        solution[0][0][tour_index] = proposed_tour

    return solution

#test ok

In [467]:
def swap_x(instance, solution, customer, neighbor, customer_tour_index):
    customer_index = [customer_tour_index, next((index for index, c in enumerate(solution[0][0][customer_tour_index]) if c == customer), -1)]
    neighbor_index = find_customer_in_vehicle_tours(solution[0][1], neighbor)


    modified_truck_tour = copy.deepcopy(solution[0][0][customer_tour_index])
    modified_truck_tour[customer_index[1]] = neighbor

    modified_drone_tour = copy.deepcopy(solution[0][1][neighbor_index[0]])
    modified_drone_tour[neighbor_index[1]] = customer

    if(is_truck_tour_feasible(instance, modified_truck_tour) and is_drone_tour_feasible(instance, modified_drone_tour)):
        solution[0][0][customer_index[0]] = modified_truck_tour
        solution[0][1][neighbor_index[0]] = modified_drone_tour

    return solution

In [468]:
def shift_t(instance, solution, customer, customer_tour_index):
    if (is_drone_eligible(instance, solution[0][1], customer)):
        solution[0][0][customer_tour_index].remove(customer)
        chosen_drone = largest_spatial_slack_drone(instance, solution[0][1])
        solution[0][1][chosen_drone].append(customer)

    return solution


In [469]:
def shift_d(instance, solution, customer, customer_tour_index):
     
    new_solution = copy.deepcopy(solution)
    new_solution[0][1][customer_tour_index].remove(customer)

    pos_best = None
    feasible_truck_tour_positions = feasible_truck_tour_positions_calculation(instance, solution[0][0], customer)
    for pos in feasible_truck_tour_positions: 
        if pos_best == None or (pos_best != None and cost(instance, insert_customer(customer, pos, copy.deepcopy(solution))) < cost(instance, insert_customer(customer, pos_best, copy.deepcopy(solution)))):
            pos_best = pos

        if pos_best == None:
            return solution
        
        solution = insert_customer(customer, pos_best, new_solution)
        return solution

In [470]:
def get_position(customer, tours):
    for vehicle_index in range(len(tours)): 
        for tour_index in range(len(tours[vehicle_index])):
            for customer_index in range(len(tours[vehicle_index][tour_index])):
                if tours[vehicle_index][tour_index][customer_index] == customer:
                    return [vehicle_index, tour_index, vehicle_index]
                
    return None

In [471]:
def is_swap_possible(instance, tours, customer, position):
    new_tour = copy.deepcopy(tours[position[0]][position[1]])
    new_tour[position[2]] = customer
    if (position[0] == 0):
        return is_truck_tour_feasible(instance, new_tour)
    if(position[0] == 1):
        return is_drone_tour_feasible(instance, new_tour)
    
    return False

metodi generali

In [472]:
def sweep_removal_operator(instance, solution, sigma):
    c_drones = get_drone_customers(solution[0][1])
    nbSweept = random.randint(0,int(len(c_drones) * sigma))

    grad_seed = random.randint(0, 360)
    c_drones = sort_by_angular(instance.coordinates, c_drones, grad_seed)
    for c in c_drones:
        if (len(solution[1]) < nbSweept):

            #inlcude tour_index calculation inside "remove_drone_customer, params: solution[0][1], c"
            tour_index = next((index for index, drone_tour in enumerate(solution[0][1]) if c in drone_tour), -1)
            solution[0][1][tour_index], solution[1] = remove_drone_customer(solution[0][1][tour_index], solution[1], c)
                               

    return solution

In [473]:
def random_drone_customer_removal(solution, sigma):
    c_drones = get_drone_customers(solution[0][1])
    q = random.randint(0,int(len(c_drones) * sigma))
    
    
    while (len(solution[1]) < q):
        customer_to_remove = random.choice(c_drones)

        c_drones.remove(customer_to_remove)
        tour_index = next((index for index, drone_tour in enumerate(solution[0][1]) if customer_to_remove in drone_tour), -1)
        solution[0][1][tour_index], solution[1] = remove_drone_customer(solution[0][1][tour_index], solution[1], customer_to_remove)

    return solution

In [474]:
def string_removal(instance, solution, c_average_removed, L_max): 
    l_s_max = max_string_length(solution[0][0], L_max)
    k_s = n_strings_to_remove(c_average_removed, l_s_max)
    
    c_seed = random.randint(1, instance.N)

    R = []

    c_adj = sort_by_euclidean_distance(instance.distances, c_seed)

    for c in c_adj:
        if len(R) < k_s:
            if any(c in truck_tour for truck_tour in solution[0][0]):
                if c not in solution[1]:
                    tour_index = next((index for index, truck_tour in enumerate(solution[0][0]) if c in truck_tour), -1)
                    if tour_index not in R:
                        l = string_to_remove_length(l_s_max, solution[0][0][tour_index])
                        
                        solution[0][0][tour_index], solution[1] = remove_string(solution[0][0][tour_index], l, c, solution[1])
                        R.append(tour_index)
            elif any(c in drone_tour for drone_tour in solution[0][1]):
                if c not in solution[1]:
                    tour_index = next((index for index, drone_tour in enumerate(solution[0][1]) if c in drone_tour), -1)
                    solution[0][1][tour_index], solution[1] = remove_drone_customer(solution[0][1][tour_index], c)


    return solution

In [475]:
def recreate(instance, solution, w1, w2, w3, w4, w5, gamma):
    solution[1] = sort_absent_customers(instance, solution, w1, w2, w3, w4, w5)
    while len(solution[1]) > 0:
        c = solution[1][0]
        pos_best = None
        
        feasible_truck_tour_positions = feasible_truck_tour_positions_calculation(instance, solution[0][0], c)
       
        for pos in feasible_truck_tour_positions: 
            if pos_best == None or (pos_best != None and cost(instance, insert_customer(c, pos, copy.deepcopy(solution))) < cost(instance, insert_customer(c, pos_best, copy.deepcopy(solution))) and random.random() > (1 - gamma)):
                pos_best = pos
        
        if is_drone_eligible(instance, solution[0][1], c):
            chosen_drone = largest_spatial_slack_drone(instance, solution[0][1])
            if pos_best == None or (pos_best != None and cost(instance, insert_customer(c, [1, chosen_drone, 0], copy.deepcopy(solution))) < cost(instance, insert_customer(c, pos_best, copy.deepcopy(solution))) and random.random() > (1 - gamma)):
                pos_best = [1, chosen_drone, 0]

        if pos_best == None:
            empty_truck_tour_index = next((index for index, truck_tour in enumerate(solution[0][0]) if len(truck_tour) == 0), -1)
            if empty_truck_tour_index == -1:
                return solution
            
            pos_best = [0, empty_truck_tour_index, 0]
        
        solution = insert_customer(c, pos_best, solution)
        solution[1].remove(c)

    return solution

In [476]:
def ruin_and_recreate(instance, solution, sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma):
    r = random.random()
    if (r < 0.5):
        solution = sweep_removal_operator(instance, solution, sigma)
    else:
        solution = random_drone_customer_removal(solution, sigma)

    solution = string_removal(instance, solution, c_average_removed, L_max)

    solution = recreate(instance, solution, w1, w2, w3, w4, w5, gamma)

    return solution


In [477]:
def local_search(instance, solution, n_nearest):
    interrupt_truck_tours = False
    interrupt_truck_customers = False

    interrupt_drone_tours = False
    #interrupt_drone_customers = False


    for i in range(len(solution[0][0])):
        
        for customer in solution[0][0][i]:
        
            neighbors = select_nearest_neighbors(n_nearest, instance.distances[customer])
            for neighbor in neighbors:
                if (neighbor in get_truck_customers(solution[0][0])):
                    neighbor_index = next((j for j, node in enumerate(solution[0][0][i]) if node == neighbor), None)
                    if neighbor_index == None:
                        new_solution = _2_opt_x(instance, copy.deepcopy(solution), customer, neighbor, i)
                        if cost(instance, new_solution) < cost(instance, solution):
                            solution = local_search(instance, new_solution, n_nearest)
                            interrupt_truck_customers = True
                            break
                    else: 
                        new_solution = relocate(instance, copy.deepcopy(solution), customer, neighbor, i)
                        if cost(instance, new_solution) < cost(instance, solution):
                            solution = local_search(instance, new_solution, n_nearest)
                            interrupt_truck_customers = True
                            break

                        new_solution = swap(instance, copy.deepcopy(solution), customer, neighbor, i)
                        if cost(instance, new_solution) < cost(instance, solution):
                            solution = local_search(instance, new_solution, n_nearest)
                            interrupt_truck_customers = True
                            break

                        new_solution = _2_opt(instance, copy.deepcopy(solution), customer, neighbor, i)
                        if cost(instance, new_solution) < cost(instance, solution):
                            solution = local_search(instance, new_solution, n_nearest)
                            interrupt_truck_customers = True
                            break

                else:
                    new_solution = swap_x(instance, copy.deepcopy(solution), customer, neighbor, i)
                    if cost(instance, new_solution) < cost(instance, solution):
                        solution = local_search(instance, new_solution, n_nearest)
                        interrupt_truck_customers = True
                        break

            new_solution = shift_t(instance, copy.deepcopy(solution), customer, i)
            if cost(instance, new_solution) < cost(instance, solution):
                solution = local_search(instance, new_solution, n_nearest)
                interrupt_truck_tours = True
                break

            if interrupt_truck_customers:
                interrupt_truck_tours = True
                break

        if interrupt_truck_tours:
            break

    for i in range(len(solution[0][1])):
       
        for customer in solution[0][1][i]:
            new_solution = shift_d(instance, copy.deepcopy(solution), customer, i)
            if cost(instance, new_solution) < cost(instance, solution):
                solution = local_search(instance, new_solution, n_nearest)
                interrupt_drone_tours = True
                break
        
        if interrupt_drone_tours:
            break
            

In [478]:
def initial_solution_construction(instance, w1, w2, w3, w4, w5, gamma, n_nearest):
    A = [c for c in range (1, instance.N)]
    solution = recreate(instance, [[[[] for _ in range(instance.h)], [[] for _ in range(instance.D)]],A], w1, w2, w3, w4, w5, gamma)
    solution = local_search(instance, solution, n_nearest)
    return solution

In [479]:
def perturbate(instance, solution, p_min, p_max, max_unfeasible_swaps):
    p = random.randint(p_min, p_max)
    swaps_executed = 0
    unfeasible_swaps = 0
    while (swaps_executed < p and unfeasible_swaps < max_unfeasible_swaps):
        c1 = random.randint(1, instance.N)
        c2 = random.randint(1, instance.N)
        pos1 = get_position(c1, solution[0])
        pos2 = get_position(c2, solution[0])
        if (is_swap_possible(instance, solution[0], c1, pos2) and is_swap_possible(instance, solution[0], c2, pos1)):
            solution[0][pos1[0]][pos1[1]][pos1[2]] = c2
            solution[0][pos2[0]][pos2[1]][pos2[2]] = c1
            swaps_executed += 1
            unfeasible_swaps = 0
        else:
            unfeasible_swaps += 1

In [480]:
def SISSRs(instance, sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma, n_nearest, delta, epsilon, iter_imp, iter_max, p_min, p_max, max_unfeasible_swaps_perturb):
    s_0 = initial_solution_construction(instance, w1, w2, w3, w4, w5, gamma, n_nearest)
    s_curr = s_0
    s_best = s_0
    iterations_without_improvement = 0
    iteration_counter = 0
    while (iteration_counter < iter_max):
        s = ruin_and_recreate(instance, copy.deepcopy(s_curr), sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma)
        if cost(instance, s) < cost(instance, s_curr)*(1+delta):
            s_curr = local_search(instance, s, n_nearest)
            if cost(instance, s_curr) < cost(instance, s_best):
                s_best = s_curr
                iterations_without_improvement = 0
            else:
                iterations_without_improvement += 1
        if iterations_without_improvement >= iter_imp:
            s_curr = perturbate(instance, s_curr, p_min, p_max, max_unfeasible_swaps_perturb)
            iterations_without_improvement = 0
        delta = delta * epsilon
        iteration_counter+=1

    return s_best

runniamo

In [481]:
SISSRs(PDSVRP_instance.PDSVRPInstance("instances/30-c-0-c.txt"), 0.3, 4.5, 4.5, 5,1,1,2,2, 0.1, 20, 0.1, 0.999975, 10, 100, 3, 5, 6)

truc travel times 31 31
tour[i] 27
distanze da i [0.35968397 0.1722353  0.54509377 0.54989656 0.00307318 0.26189587
 0.18462515 0.18197955 0.16333878 0.24695344 0.17304752 0.17922611
 0.18135202 0.17237298 0.17815412 0.15194224 0.17364107 0.54890244
 0.64311508 0.54592897 0.52179285 0.55239851 0.55221835 0.54813877
 0.54131712 0.04691008 0.01004988 0.         0.02975642 0.00975819
 0.02079797]
truc travel times 31 31
tour[i] 10
distanze da i [0.25428768 0.04148628 0.49111053 0.3873329  0.17024035 0.09055385
 0.02456058 0.06274109 0.04872257 0.08157682 0.         0.04311355
 0.01033871 0.03918191 0.00589727 0.03771973 0.04040077 0.49895201
 0.57986531 0.49163277 0.35838108 0.38708598 0.38939055 0.38563656
 0.37974919 0.13239167 0.16945534 0.17304752 0.17191341 0.17180157
 0.17602525]
truc travel times 31 31
tour[i] 10
distanze da i [0.25428768 0.04148628 0.49111053 0.3873329  0.17024035 0.09055385
 0.02456058 0.06274109 0.04872257 0.08157682 0.         0.04311355
 0.01033871 0.03918191 

IndexError: index 2 is out of bounds for axis 0 with size 0